In [1]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

In [11]:
model = ChatOpenAI(model='gpt-4.1-mini', temperature=0.0)
class Strength(BaseModel):
    """Record ys, uts, strain for material"""
    composition: Optional[str] = Field(description="nominal composition, e.g. CoCrFeNiMn0.2. Do not give if you cannot find")
    label: Optional[str] = Field(description='The label which used to uniquely identify material. Materials which undergoes different processing route should be treated as different material.')
    ys: float = Field(description="yield strength in MPa")
    uts: float = Field(description="ultimate tensile strength in MPa")
    strain: float = Field(description="strain")
    # processes: list[str] = Field(description="""list of processing steps chronological order, for each as a python dictionary. for example [{"induction melting": {"temperature": 1500}}, {"annealed": {"temperature": 800, "duration": "1h"}}]""")
    original_text: str = Field(description="original text/table contains information, which must contains the description with respect to the material been extract. This is used for later stage of associating corresponds processing history.")

class Records(BaseModel):
    records: Optional[list[Strength]]


In [10]:
text = """The HEA with a nominal composition of V10Cr15Mn5Fe35Co10Ni25 (at%) was fabricated using vacuum induction melting furnace using pure elements of V, Cr, Mn, Fe, Co, and Ni (purity >99.9%). The as-cast sample was subjected to homogenization heat treatment at 1100 °C for 6 h under an Ar atmosphere, followed by water quenching. The homogenized sample was cold rolled through multiple passes with a final rolling reduction ratio of ≈79% (from 6.2 to 1.3 mm). The disk-shaped samples (10 mm diameter) were prepared from the cold rolled sheet using electro-discharge machining. The disk samples were annealed at two different conditions (900 °C for 10 min and 1100 °C for 60 min) to obtain microstructure with fine grains and coarse grains, respectively. Finally, the HPT process was carried out on the annealed samples at different turns (N = 1/4, 1, and 5) using a pressure of 6 GPa and a rotation rate of 1 revolution per minute (rpm).
Results
Table 1. YS and UTS, and total elongation to failure (δ) of the FG and CG samples of V10Cr15Mn5Fe35Co10Ni25 HEA before (N = 0) and after HPT processing with increasing the number of turns
Number of HPT turns [N]	Sample	YS [MPa]	UTS [MPa]	δ [%]
0	FG	430	    720	    48.1
    CG	230	    532	    57.6
1/4	FG	1120	1447	15.9
    CG	1270	1502	17.3
1	FG	1630	1813	12.9
    CG	1660	1854	14.3
5	FG	1940	1986	6.0
    CG	1950	2015	6.3

Find all material information pertained to strength (ys, uts, strain) and correlated processing routes.
Here are some general rules you can apply when finding the property relevant synthesis routes from experimental section:
    1. To distinguish between materials which with same composition but different processing history, the author use unique label or relevant process parameter to describe them. Find these words in the property description section and relate with processing routes.
    2. Be cautious about the parameter variants like temperature, duration, and atmosphere etc.
Here are some synthesis method templates as reference and not all templates will be used, only the relevant ones.
induction melting: {atmosphere: "", remelting times: ""}
homogenized: temperature: {"", duration: "", atmosphere: ""}
annealed: temperature: {"", duration: "", atmosphere: ""}
cold rolled: reduction: {"", temperature: ""}
high pressure torsion: {pressure: "", turns: "", rotation speed: ""}
"""

property_text = """Table 1. YS and UTS, and total elongation to failure (δ) of the FG and CG samples of V10Cr15Mn5Fe35Co10Ni25 HEA before (N = 0) and after HPT processing with increasing the number of turns
Number of HPT turns [N]	Sample	YS [MPa]	UTS [MPa]	δ [%]
0	FG	430	    720	    48.1
    CG	230	    532	    57.6
1/4	FG	1120	1447	15.9
    CG	1270	1502	17.3
1	FG	1630	1813	12.9
    CG	1660	1854	14.3
5	FG	1940	1986	6.0
    CG	1950	2015	6.3"""

NCTB_text = """The tensile behavior of HL-NCATB before and after aging are presented in Fig. 2 a. In the engineering stress-strain curves of NCATB specimens, NCATB-2 can attain an ultimate tensile strength (UTS) of ~1.4 GPa with total elongation strain of ~ 30%, which gives the best combination of strength and ductility. SEM micrographs of the tensile fracture surfaces of NCATB-2 are presented in Fig. 2b and c, which exhibit clear evidence of transgranular ductile fracture. The selected areas with higher magnification show that the fracture surface of this sample is slanted at roughly 45° to the tensile direction, which is related to the plane of maximum shear stress and contains ductile microvoids. On the other hand, NCATB-5 shows a yield strength of ~1.1 GPa and an ultrahigh UTS of ~1.7 GPa with fracture elongation over 10%. The related SEM micrographs for the tensile fracture surfaces of NCATB-5 are presented in Supplementary Fig. S3, which also show transgranular fracture but contain larger ductile microvoids. In comparison with another solutionized NCATB specimen (annealed at 1050 °C for 30 min, composed of ultrafine grains + fine grains, UTS ~1.05 GPa, fracture elongation ~23%, microstructure presented in Supplementary Fig. S1) showing the highest UTS in our previous study [27], the NCATB-2 sample shows ~25% increase both in UTS and total elongation. Considering the solvus temperature of γ′ precipitates in the current matrix is over 950 °C [38,39], both NCATB-2 and NCATB-5 may lead to interesting intermediate temperature applications, depending on other features, such as precipitates and grain boundary stability.
Find all material information pertained to strength (ys, uts, strain) included referenced ones used for comparison"""



In [ ]:
model.with_structured_output(Records, method='json_schema').invoke(property_text).records

[Strength(composition='V10Cr15Mn5Fe35Co10Ni25', label='FG, N=0', ys=430.0, uts=720.0, strain=48.1, original_text='YS and UTS, and total elongation to failure (δ) of the FG sample of V10Cr15Mn5Fe35Co10Ni25 HEA before HPT processing (N=0)'),
 Strength(composition='V10Cr15Mn5Fe35Co10Ni25', label='CG, N=0', ys=230.0, uts=532.0, strain=57.6, original_text='YS and UTS, and total elongation to failure (δ) of the CG sample of V10Cr15Mn5Fe35Co10Ni25 HEA before HPT processing (N=0)'),
 Strength(composition='V10Cr15Mn5Fe35Co10Ni25', label='FG, N=1/4', ys=1120.0, uts=1447.0, strain=15.9, original_text='YS and UTS, and total elongation to failure (δ) of the FG sample of V10Cr15Mn5Fe35Co10Ni25 HEA after HPT processing with 1/4 turn'),
 Strength(composition='V10Cr15Mn5Fe35Co10Ni25', label='CG, N=1/4', ys=1270.0, uts=1502.0, strain=17.3, original_text='YS and UTS, and total elongation to failure (δ) of the CG sample of V10Cr15Mn5Fe35Co10Ni25 HEA after HPT processing with 1/4 turn'),
 Strength(composit

In [12]:
model.with_structured_output(Records, method='json_schema').invoke(NCTB_text).records

[Strength(composition=None, label='NCATB-2', ys=1400.0, uts=1400.0, strain=0.3, original_text='NCATB-2 can attain an ultimate tensile strength (UTS) of ~1.4 GPa with total elongation strain of ~ 30%'),
 Strength(composition=None, label='NCATB-5', ys=1100.0, uts=1700.0, strain=0.1, original_text='NCATB-5 shows a yield strength of ~1.1 GPa and an ultrahigh UTS of ~1.7 GPa with fracture elongation over 10%'),
 Strength(composition=None, label='Solutionized NCATB (annealed at 1050 °C for 30 min)', ys=1050.0, uts=1050.0, strain=0.23, original_text='solutionized NCATB specimen (annealed at 1050 °C for 30 min, composed of ultrafine grains + fine grains, UTS ~1.05 GPa, fracture elongation ~23%)')]